# AWS CloudFormation

## What is AWS CloudFormation

> AWS CloudFormation is an AWS service that helps you setup and manage your AWS resources and services infastructure. It allows you to design the **Infastructure as Code** allowing you more time to focus on building your AWS applications. By defining a YAML or JSON template you can describe all the AWS resources required(like EC2, AWS RDS or S3) by your application and CloudFormation will handle the provisioning of those resources for you. 

For full details on what is possible with AWS CloudFormation consult the documentation [here](https://docs.aws.amazon.com/cloudformation/index.html).

### Benefits of using CloudFormation

* **Simplifies creating your application infastructure:** Imagine your application infastruture was to deploy an EC2 instance and S3 bucket with the correct security permissions. Doing this manually can be very time consuming, by defining the process in a template you can quickly create another instance of this infastructure in a few clicks.
* **Replicate your architecture:** Since your CloudFormation infastructure is defined in code by your template, you can easily reuse the template to deploy the same infastruture to another AWS region.
* **Track changes easily:** If you want to make changes to your infastructure for instance updating security permission to your EC2 instance. If a problem occured while doing this then you would need to remember the previous settings. CloudFormation provides a rollback feature to use a previous version of your template if a problem occurs and tracks the changes made.
* **Free to use:** CloudFormation is fully free to use but be aware that the services your define in your template will have the normal associated costs. Each template does have limited quotas when being created and the quotas can be viewed [here](https://docs.aws.amazon.com/cloudformation/index.html).

### CloudFormation Concepts

* **Template:** Template can be specified in JSON or YAML format and act as the blueprint for your AWS infrastructure. It will define the resources used and how they are configured when added to the stack.
* **Stack:** After configuring your AWS template and deploying it your resources will be managed in a single object called a stack. From the stack you can update, delete, create resources to your stack defined by the template. 
* **Change sets:** Allow you to generate a summary of proposed changes which will be made to your stack before updating it. Since AWS destroys the previous stack when updating a new one you can view the changes to ensure no critcal elements are affected when updating the new stack. 



## Creating CloudFormation Templates

Before you can begin to use the CloudFormation we will need to design a template that we can upload to create our stack. We have the option to create the template in YAML and JSON, we will be using YAML for the purposes of this notebook. The YAML template is where you will configure your AWS resources to be added to your stack. The full avaliable structure of an AWS CloudFormation template can be found [here](https://docs.aws.amazon.com/AWSCloudFormation/latest/UserGuide/template-anatomy.html).

Below is a basic template with defines a simple AWS CloudFormation stack configuring an EC2 instance. The general process of creating a CloudFormation template will be to define the AWS resources that your CloudFormation stack will contain and then define the properties for each of those resources. In the example we have defined a new Unbuntu EC2 instance, notice the properties of the resource(ImageId, InstanceType and Amazon EBS storage) make sense for an EC2 instance.  

Some properties for your resource will be required, have defaults or be fully optional, to understand which properties and what resources are avaliable for your template. You can consult the resource and property reference of the CloudFormation documentation [here](https://docs.aws.amazon.com/AWSCloudFormation/latest/UserGuide/aws-template-resource-type-ref.html).



In [ ]:
#Date and describe your CloudFormation template
AWSTemplateFormatVersion: '16-09-2021'                  
Description: 'Your first AWS CloudFormation template'

# Define a resource dictionary which will contain our AWS resources
Resources:
# Name our resource so it can be referenced if needed                                 
  UbuntuServer20.04:
    Type: "AWS::EC2::Instance"
    # Define the properties for the resource in a new dictionary
    Properties:
                              
      AccessControl: PublicRead
      ImageId: "ami-0a8e758f5e873d1c1"     
      InstanceType: t2.micro  
      # input your own AWS key  
      KeyName: yourkey            
      BlockDeviceMappings:                  
        -
          DeviceName: /dev/sda1
          Ebs:
            VolumeType: standard
            VolumeSize: 5
            DeleteOnTermination: True
            Encrypted: False


Let's create another resource for our template that will allow use to SSH into our EC2 instance when created. To do this we will configure an EC2 security group. 

In [ ]:
# Date and describe your CloudFormation template
AWSTemplateFormatVersion: "2010-09-09"                    
Description: "Your first AWS CloudFormation template"

# Define a resource dictionary which will contain our AWS resources
Resources: 
# Name our resource so it can be referenced if needed                                 
  UbuntuServerInstance:                       
    Type: "AWS::EC2::Instance"
    # Define the properties for the resource in a new dictionary
    Properties:
      # Reference the security group we created so our EC2 instance will be preconfigured with SHH access          
      ImageId: "ami-0a8e758f5e873d1c1"     
      InstanceType: t2.micro  
      # input your own AWS key  
      KeyName: yourkeyname               
      BlockDeviceMappings:                  
        -
          DeviceName: /dev/sda1
          Ebs:
            VolumeType: standard
            DeleteOnTermination: True
            Encrypted: False
        
# Define the new Security group resource
  EC2instanceSecurityGroup:
    # Name the resource so we can reference it                 
    Type: "AWS::EC2::SecurityGroup" 
    # Configure the resources properties        
    Properties:                             
      GroupDescription: Allow SSH Access to an EC2 instance on port 22      
      SecurityGroupIngress:
        - IpProtocol: tcp
          FromPort: 22
          ToPort: 22
          CidrIp: 0.0.0.0/0

### The Ref function 

Excellent we have configured the new resource but wait there is a problem, how will the EC2 instance we have created know to use the security group we just created? The reference function is one of the many functions you apply in any of your templates, there are many others and can be found [here](https://docs.aws.amazon.com/AWSCloudFormation/latest/UserGuide/intrinsic-function-reference.html). You can do this using the reference function, referencing resources or properties can be done using the function like so.

In [ ]:
# In JSON 
# Logical name wil refer to the resource or property you want to refer to.
{ "Ref" : "LogicalName" } 

# In YAML
# Long form
Ref : LogicalName
#Short form
!Ref LogicalName

Lets now add this to our template so the EC2 instance will be preconfigured to work with SSH when created, the EC2 resource has a security group property so we can reference the secuirty group there.

In [ ]:
# Date and describe your CloudFormation template
AWSTemplateFormatVersion: "2010-09-09"
Description: "Your first AWS CloudFormation template"

# Define a resource dictionary which will contain our AWS resources
Resources: 
# Name our resource so it can be referenced if needed                                 
  UbuntuServerInstance:            
    Type: "AWS::EC2::Instance"
    # Define the properties for the resource in a new dictionary
    Properties:
      # Reference the security group we created so our EC2 instance will be preconfigured with SHH access
      SecurityGroups:
        - !Ref EC2instanceSecurityGroup                
      ImageId: "ami-0a8e758f5e873d1c1"     
      InstanceType: t2.micro  
      # input your own AWS key  
      KeyName: yourkeyname               
      BlockDeviceMappings:                  
        - 
        DeviceName: /dev/sda1
          Ebs:
            VolumeType: standard
            DeleteOnTermination: True
            Encrypted: False
        
# Define the new Security group resource
  EC2instanceSecurityGroup:
    # Name the resource so we can reference it                 
    Type: "AWS::EC2::SecurityGroup" 
    # Configure the resources properties        
    Properties:                             
      GroupDescription: Allow SSH Access to an EC2 instance on port 22      
      SecurityGroupIngress:
        - IpProtocol: tcp
          FromPort: 22
          ToPort: 22
          CidrIp: 0.0.0.0/0

## Passing parameters in your template

Next notice in our template the EC2 instance has a property **KeyName** which requires a value. Now it wouldn't be best practice to specify this value directly in our template for security reasons, so how do we get around this? We can create custom parameters which the user will need to define when creating the stack in CloudFormation. Each parameter is declared with a list of it's attributes and a required type, which can be a string, number or an AWS type and we can add a description for the parameter. Let's add KeyName as a parameter and reference it in our EC2 instance so it will be required when creating our stack.

In [ ]:
# Date and describe your CloudFormation template
AWSTemplateFormatVersion: "2010-09-09"                    #
Description: "Your first AWS CloudFormation template"

# Create the parameters entry and define our parameters here.
Parameters:
  # Parameter for the user key name pair on stack creation
  UsersKeyName:
    Description: Name of a key name pair that exists on your AWS account.
    Type: AWS::EC2::KeyPair::KeyName


# Define a resource dictionary which will contain our AWS resources
Resources: 
# Name our resource so it can be referenced if needed                                 
  UbuntuServerInstance:                       
    Type: "AWS::EC2::Instance"
    # Define the properties for the resource in a new dictionary
    Properties:
      # Reference the security group we created so our EC2 instance will be preconfigured with SHH access
      SecurityGroups:
        - !Ref EC2instanceSecurityGroup                
      ImageId: "ami-0a8e758f5e873d1c1"     
      InstanceType: t2.micro  
      # input your own AWS key  
      KeyName: !Ref UsersKeyName               
      BlockDeviceMappings:                  
        -
          DeviceName: /dev/sda1
          Ebs:
            VolumeType: standard
            DeleteOnTermination: True
            Encrypted: False
            
  # Define the new Security group resource
  EC2instanceSecurityGroup:
    # Name the resource so we can reference it                 
    Type: "AWS::EC2::SecurityGroup" 
    # Configure the resources properties        
    Properties:                             
      GroupDescription: Allow SSH Access to an EC2 instance on port 22      
      SecurityGroupIngress:
        - IpProtocol: tcp
          FromPort: 22
          ToPort: 22
          CidrIp: 0.0.0.0/0


Finally before finishing the creation of our stack let's add an Amazon RDS service, S3 bucket and configure a Amazon IAM role that will allow the creation and access to these resources. Once you have finished creating your template save it in a YAML file named **Ourfirststack.yaml** ready to be uploaded. For the database password parameters notice **NoEcho** is used, this allows the password to be set and viewed within the stack using asteriks adding security to the password created. 

In [ ]:
# Date and describe your CloudFormation template
AWSTemplateFormatVersion: "2010-09-09"                    #
Description: "Your first AWS CloudFormation template"

# Create the parameters entry and define our parameters here.
Parameters:
  # Parameter for the user key name pair on stack creation
  UsersKeyName:
    Description: Name of a key name pair that exists on your AWS account.
    Type: AWS::EC2::KeyPair::KeyName

  # Parameter to set the RDS database password on creation
  MySQLDBPassword: 
    Description: Parameter to configure the master password to our postgres database on creation
    Type: String
    NoEcho: true
  
  # Parameter to set the bucketname on creation
  BucketName:
    Description: Parameter to give unique name to S3 bucket
    Type: String

# Define a resource dictionary which will contain our AWS resources
Resources: 
# Name our resource so it can be referenced if needed                                 
  UbuntuServerInstance:                       
    Type: "AWS::EC2::Instance"
    # Define the properties for the resource in a new dictionary
    Properties:
      # Reference the security group we created so our EC2 instance will be preconfigured with SHH access
      SecurityGroups:
        - !Ref EC2instanceSecurityGroup                
      ImageId: "ami-0a8e758f5e873d1c1"     
      InstanceType: t2.micro  
      # input your own AWS key  
      KeyName: !Ref UsersKeyName               
      BlockDeviceMappings:                  
        -
          DeviceName: /dev/sda1
          Ebs:
            VolumeType: standard
            DeleteOnTermination: True
            Encrypted: False
            
  # Define the new Security group resource
  EC2instanceSecurityGroup:
    # Name the resource so we can reference it                 
    Type: "AWS::EC2::SecurityGroup" 
    # Configure the resources properties        
    Properties:                             
      GroupDescription: Allow SSH Access to an EC2 instance on port 22      
      SecurityGroupIngress:
        - IpProtocol: tcp
          FromPort: 22
          ToPort: 22
          CidrIp: 0.0.0.0/0

  CloudStackIAMRole:
    Type: AWS::IAM::Role
    Properties:
      Description: Create new user role to allow access to RDS instance, S3 bucket and EC2 instance created by the stack.
      RoleName: StackUser
      AssumeRolePolicyDocument:
        Version: "2012-10-17"  
        Statement:
          - Effect: Allow
            Principal:
              Service:
                - ec2.amazonaws.com
                - rds.amazonaws.com
                - s3.amazonaws.com
            Action: 
              - 'sts:AssumeRole'
      Path: "/"


  RolePolicies:
    Type: "AWS::IAM::Policy"
    Properties: 
      PolicyName: "NewStackUser"
      PolicyDocument:
        Version: "2012-10-17"
        Statement:
          - Effect: "Allow"
            Action: "*"  
            Resource: "*"
      Roles: 
        - !Ref CloudStackIAMRole

  RootInstanceProfile: 
    Type: "AWS::IAM::InstanceProfile"
    Properties: 
      Path: "/"
      Roles: 
        - Ref: CloudStackIAMRole

  AWSS3Bucket:
    Type: AWS::S3::Bucket
      # Specify when the stack is deleted to keep the bucket
    DeletionPolicy: Retain
    Properties: 
      # Add the BucketName parameter so that a bucket name is required on stack creation
      BucketName: !Ref BucketName

  AWSRDSDatabase:
    Type: AWS::RDS::DBInstance 
    Properties:
      DBName: templateDB
      Engine: MySQL
      MasterUsername: admin
      MasterUserPassword: !Ref MySQLDBPassword
      PubliclyAccessible : true
      DBInstanceClass: db.t2.micro
      AllocatedStorage: 20


## Creating the stack

Now that we have our stack infrastructure defined in code we can now upload the template to CloudFormation to create our stack, go to the CloudFormation page on AWS services. Once on the CloudFormation dashboard select **stacks > Create stack > With new resources** and upload the template we created. It is on the create stack screen if your template isn't in a valid YAML/JSON format then CloudFormation will alert you so you can make changes to your template. Once uploaded with no template errors continue to the next screen.


<img src="images/create_stack.png?modified=12345678" />



Here we can name our stack and finish defining any required parameters that we have created in our YAML template. Finish naming the stack and finish adding the values for the parameters which were required in our YAML template. 

<img src="images/stack_name_parameters.png?modified=123324678"/>


On the next page your can configure addtional options for your stack such as rolling back the whole stack if there is a failure or saving the resources that were created successfully. You can define a custom role that will be responsible for changing the stack in anyway and tag resources in your stack to more easily categorise and identify them. There are other more advanced options that I will leave you to investigate but for now we will leave this screen with the defaults and continue.

<img src="images/config_stack.png?modified=12345678" />

Lastly we can review we the stack we have created with the options we have defined previously. Since we are creating a stack that defines a IAM role you will have to acknowledge that the stack will create this role you will be met with the following panel at the bottom of the review screen. Since we are happy for our stack to create the IAM role achknowlege this option and finish creating the stack.

<img src="images/IAM_role_warning.png?modified=1345678" />

Once we begin creating our stack we will be taken to the created stacks event page where the stack will begin being created. After sometime the events will update with each individual step needed to create the stack. Once our stack is full created and functioning the stack creation progress will change from **CREATE_IN_PROGESS** to **CREATE_COMPLETE**. Once completed take the time to view the various tabs across the top to view in more detail your stacks configuration. Also check that the AWS services were created by the stack we created using the template.

<img src="images/stack_overview.png?modified=1345678" />

## Creating a stack change set

Now that we know how to create a stack defined in a template let's now make a change set by editing our template and uploading to see the effect of the change on our template. Copy your template rename **Changesettemplate.yaml** YAML template by changing the type of database engine, deletion policy of the S3 bucket and remove SSH access from our EC2 instance. 


In [ ]:
# Date and describe your CloudFormation template
AWSTemplateFormatVersion: "2010-09-09"                    #
Description: "Your first AWS CloudFormation template"

# Create the parameters entry and define our parameters here.
Parameters:
  # Parameter for the user key name pair on stack creation
  UsersKeyName:
    Description: Name of a key name pair that exists on your AWS account.
    Type: AWS::EC2::KeyPair::KeyName

  # Parameter to set the RDS database password on creation
  MariaDBDBPassword: 
    Description: Parameter to configure the master password to our postgres database on creation
    Type: String
    NoEcho: true
  
  # Parameter to set the bucketname on creation
  BucketName:
    Description: Parameter to give unique name to S3 bucket
    Type: String

# Define a resource dictionary which will contain our AWS resources
Resources: 
# Name our resource so it can be referenced if needed                                 
  UbuntuServerInstance:                       
    Type: "AWS::EC2::Instance"
    # Define the properties for the resource in a new dictionary
    Properties:
      # Reference the security group we created so our EC2 instance will be preconfigured with SHH access               
      ImageId: "ami-0a8e758f5e873d1c1"     
      InstanceType: t2.micro  
      # input your own AWS key  
      KeyName: !Ref UsersKeyName               
      BlockDeviceMappings:                  
        -
          DeviceName: /dev/sda1
          Ebs:
            VolumeType: standard
            DeleteOnTermination: True
            Encrypted: False
            

  CloudStackIAMRole:
    Type: AWS::IAM::Role
    Properties:
      Description: Create new user role to allow access to RDS instance, S3 bucket and EC2 instance created by the stack.
      RoleName: StackUser
      AssumeRolePolicyDocument:
        Version: "2012-10-17"  
        Statement:
          - Effect: Allow
            Principal:
              Service:
                - ec2.amazonaws.com
                - rds.amazonaws.com
                - s3.amazonaws.com
            Action: 
              - 'sts:AssumeRole'
      Path: "/"


  RolePolicies:
    Type: "AWS::IAM::Policy"
    Properties: 
      PolicyName: "NewStackUser"
      PolicyDocument:
        Version: "2012-10-17"
        Statement:
          - Effect: "Allow"
            Action: "*"  
            Resource: "*"
      Roles: 
        - !Ref CloudStackIAMRole

  RootInstanceProfile: 
    Type: "AWS::IAM::InstanceProfile"
    Properties: 
      Path: "/"
      Roles: 
        - Ref: CloudStackIAMRole

  AWSS3Bucket:
    Type: AWS::S3::Bucket
      # Specify when the stack is deleted to keep the bucket
    DeletionPolicy: Delete
    Properties: 
      # Add the BucketName parameter so that a bucket name is required on stack creation
      BucketName: !Ref BucketName

  AWSRDSDatabase:
    Type: AWS::RDS::DBInstance 
    Properties:
      DBName: templateDB
      Engine: MariaDB
      MasterUsername: admin
      MasterUserPassword: !Ref MariaDBDBPassword
      PubliclyAccessible : true
      DBInstanceClass: db.t2.micro
      AllocatedStorage: 20

After making these changes select the **Change sets** tab for our stack and **Create change set**.  Now on the create change set screen upload our newly created template as a replacement and continue to the parameters screen.

<img src="images/change_set_template.png?modified=145678" />

On the next screen the change set will allow us the option of editing the existing parameters if we so choose. In this case leave the parameters as the previously set values and confirm the details on the confguration and review stages of the change set creation.

<img src="images/change_set_parameters.png?modified=1611568" />

Once the review stage is complete the last thing you will need to do is give a description for your change set. Be sure to give the change set a detailed description so that any other users viewing the stacks details will know which changes were implemented on the stack. 

<img src="images/change_set_desc.png?modified=1611568" />

Once you have completed the description and created the change set the next step is to view the changes made and execute the change if we are happy with the results. We can see that our change set has completed successfully and the changes to the stack are what we expected. The EC2 policy is being removed and there is modifications to the RDS database and EC2 instance. 

We can also check the changes made to the template using the Template tab along the top of the **Changes** pane. Once you are happy with the changes you can implement them using the **Execute** button(top right) with the option to remove succesfully provisioned resources or roll back the entire stack, for our purposes we will roll back the entire stack.

<img src="images/change_set_finalisation.png?modified=1611568" />

Once executed you will be taken to the events screen to view the progress of your change set being implemented. If all goes well then your stack will finish completing with the status **UPDATE_COMPLETE**. Great now we know how to be made aware of the changes to our stack so we change safely implement them when editing our template. 

## Summary

* You have learned to define your infastruture in code using templates and why this process is better than manual configuration.
* You can now create templates which define your infastucture in code. 
* You know the basic layouts of templates and how to configure it's resources. 
* We know how to create parameters for our template to add customisation to our stack.
* We learned how change sets are useful when making changes to our stack so when we change our stack it is done safely. 